# 2 Numerical computation
* [**2.0 Preable**](#2.0-Preamble)
* [**2.1 Vectorized thinking**](#2.1-Vectorized-thinking)
* [**2.2 Arrays**](#2.2-Arrays)
* [**2.3 Numerical optimization and fitting**](#2.3-Numerical-optimization-and-fitting)
* [**2.4 Simulation**](#2.4-Simulation)
* [**More exercises**](#More-exercises)

Working with numbers is central to almost all scientific and engineering computing, from deep learning to image processing to climate simulation. We could use Python directly for numerical
computation&mdash;but it's much faster to use Python just as 'glue', using it to write concise code and to quickly develop our thinking, and to rely on 
carefully optimized low-level libraries for the heavy lifting.

<div style="background-color:wheat">
<strong>Goal of this notebook.</strong>
Learn your way around the two main Python libraries for numerical work, 
<a href="http://www.numpy.org/">NumPy</a> 
and 
<a href="https://www.scipy.org/">SciPy</a>,
learn how to produce basic plots with [matplotlib](https://matplotlib.org/).

If you have come across NumPy before, or if you like to pick up programming by working on problems rather than reading tutorials, skim through the section on <a href="#2.1-Vectorized-thinking">vectorized thinking</a> first and then jump ahead to 
<a href="Assignment%202.ipynb">Assignment 2</a>.
</div>

## 2.0 Preamble

At the top of almost every piece of scientific computing work, we'll import these standard modules. 

In [ ]:
# Import modules, and give them short aliases so we can write e.g. np.foo rather than numpy.foo
import math, random
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.optimize
import pandas
# The next line is a piece of magic, to let plots appear in our Jupyter notebooks
%matplotlib inline    

## 2.1 Vectorized thinking

We have already seen Python lists, which can store mixed data types e.g. integers mixed with strings and sublists and even functions. The flexibility comes at the price of performance. In scientific computing, it's better to use specialised classes for vectors, and to use functions that operate on entire vectors at once. This is called _vectorized thinking_, and it's a core skill for scientific computing. Once you get the hang of it, you will write code that is more concise and faster.

Here's an example: computing the [correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between two vectors $x$ and $y$,

$$
\rho
= 
\frac{\sum_i(x_i - \bar{x})(y_i-\bar{y})}
{\sqrt{\sum_i(x_i-\bar{x})^2)} \sqrt{\sum_i(y_i-\bar{y})^2}}
$$

where $x$ and $y$ have the same length $N$, and 

$$
\bar{x}=\frac{1}{N}\sum_i x_i,
\quad
\bar{y}=\frac{1}{N}\sum_i y_i.
$$

Here are two pieces of code, one written in Python-style, one written in scientific computing style, to compute $\rho$. The latter is roughly 15 times faster. The magic command `%%time` at the top of a cell makes the notebook print out the execution time.

In [ ]:
# Set up some parameters.
# We'll use a random number seed so our code is reproducible. Python's hash function gives an integer,
# suitable for use as a seed.
N = 10000000
rand_seed = hash('geranium') % (2 << 31)

In [ ]:
%%time
# Python-style code

random.seed(rand_seed)
# Create two lists of random numbers, xs and ys, where each y depends on the corresponding x
xs = [random.random() for i in range(N)]
ys = [xs[i] + random.random() for i in range(N)]
# Compute the various terms involved in the formula
xbar = sum(xs) / N  # sum(list) is built into Python
ybar = sum(ys) / N
sxy = sum([(x-xbar)*(y-ybar) for x,y in zip(xs,ys)])  # this is how to iterate over a pair of lists
sxx = sum([(x-xbar)**2 for x in xs])
syy = sum([(y-ybar)**2 for y in ys])
rho = sxy / math.sqrt(sxx) / math.sqrt(syy)
print(rho)

In [ ]:
%%time
# Vectorized code

np.random.seed(rand_seed)
# Create two random vectors x and y
x = np.random.random(N)
y = x + np.random.random(N)
# Compute the terms in the formula. Note: @ means "dot product"
xbar = np.sum(x) / N
ybar = np.sum(y) / N
rho = ((x-xbar) @ (y-ybar)) / math.sqrt(np.sum((x-xbar)**2)) / math.sqrt(np.sum((y-ybar)**2))
print(rho)

To be good at writing vectorized code, we need to know what sort of vectorized routines there are available in the NumPy library.
(And, if we really did know our way round the library, we'd have used [`np.corrcoef`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html#numpy.corrcoef) rather than write our own code for $\rho$.)
Here are some [useful routines](https://docs.scipy.org/doc/numpy/reference/routines.html#routines).

**Create vectors:**
<ul style="margin-top: 0px">
<li>`np.array([1,2,3])` creates a numpy vector from a Python list
<li>`np.zeros(n)`, `np.ones(n)`, `numpy.full(n,fill_value)`
<li>`numpy.ones_like(a)` creates a vector of the same shape as `a`
<li>`np.arange` is like Python's `range`, `np.linspace(start,stop,n)` creates $n$ evenly-spaced points between `start` and `stop` inclusive
<li>`np.random.random(n)`, `np.random.choice(a,n)`, and other [random number generators](https://docs.scipy.org/doc/numpy/reference/routines.random.html)
<li>and other [array creation](https://docs.scipy.org/doc/numpy/reference/routines.array-creation.html#array-creation-routines) routines
</ul>

**Maths:**
<ul style="margin-top: 0px">
<li>Normal mathematical expressions work on vectors, and you can mix vectors and scalars, e.g. `x + y ** 2 + 5 >= z`
<li>`np.sin`, `np.exp`, `np.floor`, ...
<li>`x @ y` gives the dot product, `np.linalg.norm(x)` is the norm
<li>`np.sum`, `np.prod`, ..., and `np.cumsum(x)` for $[x_0, x_0+x_1, x_0+x_1+x_2, \dots]$
<li>`np.min`, ..., and `np.minimum(x,y)` for $[\min(x_0,y_0), \min(x_1,y_1), \dots]$
<li> and other [maths](https://docs.scipy.org/doc/numpy/reference/routines.math.html#mathematical-functions) and [statistics](https://docs.scipy.org/doc/numpy/reference/routines.statistics.html) functions
</ul>

**Programming:**
<ul style="margin-top: 0px">
<li>`len(x)` gives the length of a vector
<li>`np.sort`, `np.argmax` and other [search functions](https://docs.scipy.org/doc/numpy/reference/routines.sort.html#sorting)
<li>`np.any`, `np.all` and other [logic functions](https://docs.scipy.org/doc/numpy/reference/routines.logic.html#logic-functions)
<li>`np.where(cond,x,y)` is the vectorized version of `x if cond else y`
</ul>

**Indexing:**
<ul style="margin-top: 0px">
<li>The usual [slice notation](http://localhost:8888/notebooks/1__programming.ipynb#Collections:-tuples,-lists,-and-dictionaries) works, e.g. `x[:10]` or `x[10:]` or `x[:-3]`
<li>We can index by a vector of booleans, e.g. `x[y>5] = 3`
<li>We can index by a vector of integers, e.g. `i=np.argwhere(y>5); x[i]=3`
<li>`np.concatenate([v1,v2])` concatenates two or more vectors
</ul>

<div style="background-color:wheat"><strong>Exercise (ex1a).</strong>
Using Python-style code: let $x$ and $y$ be random numbers in the range $[-1,1]$, and let $d=\sqrt{x^2+y^2}$. Repeat this a million times, and find the mean and standard deviation of your values for $d$.
</div>

<div style="background-color:wheat"><strong>Exercise (ex1b).</strong>
Repeat the previous exercise, but this time using vectorized code. Compare the speed of the two styles of code.
</div>

In NumPy, all the elements of a vector have to be the same type. Use `x.dtype` to find this type,
and [`x.astype`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html#numpy-ndarray-astype) to convert a vector from one type to another.

In [ ]:
x = np.arange(10)
y = (x > 5)
z = np.ones_like(x)
(x.dtype, y.dtype, z.dtype)

## 2.2 Arrays

NumPy supports matrices and higher-dimensional arrays. (In fact, when we look up the help for any of the vectorized routines, we see that vectors are nothing more than one-dimensional arrays.) To enter a 2d array like
$$
a = \left( \begin{matrix} 2.2 & 3.7 & 9.1\\-4 & 3.1 & 1.3\end{matrix}\right)
$$
we type in

In [ ]:
a = np.array([[2.2, 3.7, 9.1], [-4, 3.1, 1.3]])

Use `a.shape` to find the dimensions of an array:

In [ ]:
a.shape

<div style="background-color:wheat"><strong>Exercise (ex2).</strong> What is the relationship between <code style="background-color:wheat">a.shape</code> and <code style="background-color:wheat">len(a)</code>?</div>

For 1d vectors the only reshaping operations are slicing and concatenating, but for higher dimensional arrays there is a whole variety of [reshaping functions](https://docs.scipy.org/doc/numpy/reference/routines.array-manipulation.html#array-manipulation-routines) such as stacking, tiling, transposing, etc.
NumPy also has a powerful tool called [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) which generalizes "add a scalar to a vector", and which is used a lot in more advanced array-manipulating code. It's more advanced than we need for this course, but it's used a lot in machine learning and it's worth reading about.

<div style="background-color:wheat"><strong>Exercise (ex3).</strong> Look up the NumPy help for <code style="background-color:wheat">arange</code> and <code style="background-color:wheat">reshape</code>, and use these functions to produce the $3\times5$ matrix
$$
b = \left( \begin{matrix} 
1 & 2 & 3 & 4 & 5\\
6 & 7 & 8 & 9 & 10\\
11 & 12 & 13 & 14 & 15
\end{matrix} \right)
$$
Look up the help for <code style="background-color:wheat">sum</code>, and compute the length-5 vector of column sums and the length-3 vector of row sums.
</div>

<div style="background-color:wheat"><strong>Exercise (ex4).</strong>
Find two different ways to use NumPy to create the column vector <code style="background-color:wheat">array([[1],[2],...,[n]])</code>.
</div>

To refer to a subarray, we can use an extended version of Python's slice notation.

In [ ]:
a[:, :2]                   # all rows, first two columns
a[1, :]                    # second row (indexes start at 0), all columns
a[1]                       # another way to fetch the second row
a[:2, :2] = [[1,2],[3,4]]  # assign to a submatrix
a

To refer to arbitrary sets of elements in the array, we can use boolean indexing e.g. `a[a>=2]`, or integer indexing as in the code snippet below. These are both called [advanced indexing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing).

In [ ]:
a = np.zeros((3,3), dtype=np.int)
a[[0,1,2], [1,0,2]] = [1,1,1]
a

<div style="background-color:wheat"><strong>Exercise (ex5).</strong>
A <a href="https://en.wikipedia.org/wiki/Permutation_matrix">permutation matrix</a> is a square matrix of 0s and 1s,
where each row contains exactly one 1, and each column likewise. (The code snippet above creates a $3\times3$ permutation matrix.) Write code to generate a random $n\times n$ permutation matrix.
</div>

In Easter term, you will study linear algebra in the _Maths for Natural Sciences_ course. If you want to try out the maths, you'll find relevant functions in [`np.linalg`](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html)
and
[`np.dual`](https://docs.scipy.org/doc/numpy/reference/routines.dual.html).

## 2.3 Numerical optimization and fitting

A common task in science and in machine learning is to find the minimum value of a function, which may have one or more variables. For example, we might have a collection of points that more or less follow a straight line, and we might want to use the equation $y=mx+c$. In this case, we'd like to tune the values of $m$ and $c$ so that the equation lies close to the data. We can achieve this by defining a function $L(m,c)$ that measures how far the points are from the straight line, and then choosing $m$ and $c$ to minimize $L(m,c)$.

<span style="background-color:red; padding:1pt; color:white">WARNING!</span> The methods we discuss here sometimes work brilliantly, but sometimes are unstable. This is not the fault of Python or the libraries we are using. 
It's just the case that sometimes the equations in the algorithm and numerical issues in the data are not well balanced. This is something we need to bear in mind every time we use these methods, and we should check the output, for example by plotting graphs.

Let's start by minimizing a simple function of one variable. We could use calculus to find the minimum for a simple example like this, but let's do it with computer power instead.

In [ ]:
def f(x, a, b, c):
    return a*x + b*(x**2) + c*(x**4)

We'll plot this function first, to get a rough idea of where the minimum should be.
Visualisation is a crucial part of scientific computing, and we'll cover it in much more detail in [&sect;3 Working with Data](3.%20Working%20with%20data.ipynb), but for present purposes we'll just use some very simple plotting commands. The [pyplot tutorial](https://matplotlib.org/users/pyplot_tutorial.html) explains more options.

In [ ]:
x = np.linspace(-2,2,40)          # 40 equally spaced points in the range [-2,2]
y = f(x, a=1, b=-3, c=1)          # f is automatically vectorized, because it only uses vectorized expressions
plt.plot(x, y, linestyle='-', linewidth=1, color='red')
plt.show()

The `scipy.optimize.fmin` function finds where the function achieves its minimum value, starting from an initial guess `x0`. The first argument is the function to optimize. In the snippet below we're giving it an anonymous function that is a version of `f` with the parameters `a`, `b` and `c` filled in.

In [ ]:
scipy.optimize.fmin(lambda x: f(x,a=1,b=-3,c=1), x0=0.5)

It found a local minimum, not the global minimum. This is often a problem with numerical optimization routines, and it's why it's helpful to look at the data first.

<div style="background-color:wheat"><strong>Exercise (ex6).</strong> 
What starting point do we need to give, so that it can find the global optimum?
Run <code style="background-color:wheat">scipy.optimize.fmin</code> for a range of values of 
<code style="background-color:wheat">x0</code> in the range $[-2,2]$. Tabulate 
<code style="background-color:wheat">x0</code> versus the minimizing $x$ it finds. You can turn off the diagnostic output with the option
<code style="background-color:wheat">disp=False</code>.</div>

Here is an example of a function of two variables. We'll try to fit the straight line $y=mx+c$ through a set of points. We'll define the _loss function_
$$
L(m,c) = \sum_i (m x_i + c - y_i)^2
$$
and look for $m$ and $c$ to minimize it.

In [ ]:
x = np.linspace(-3,3,40)
y = np.sin(x) + 2 * np.random.random(x.shape)

def loss(m,c):
    return np.sum((m*x+c - y)**2)

Before we do any numerical work, we should visualize what we're doing. Here are a plot of the points $(x_i,y_i)$, and a surface plot of the loss function $L(m,c)$ as a function of $m$ and $c$. The 3d plotting functions are somewhat mysterious, and you should look at [relevant examples](https://matplotlib.org/examples/mplot3d/index.html) and copy them. 

In [ ]:
# Scatter plot of the data
plt.plot(x, y, marker='o', markerfacecolor='white', markeredgecolor='black', linestyle='None')
plt.show()

In [ ]:
# Surface plot of loss(m,c)
# adapted from https://matplotlib.org/examples/mplot3d/surface3d_demo.html

# Create 2d arrays, one with each m value, one with each c value, one with loss
m,c = np.meshgrid(np.linspace(-3,3,20), np.linspace(-3,5,20))
l = np.zeros_like(m)
for i in range(l.shape[0]):
    for j in range(l.shape[1]):
        l[i,j] = loss(m[i,j], c[i,j])

from mpl_toolkits.mplot3d import axes3d   # import a library to allow 3d plots
axes = plt.figure().gca(projection='3d')  # say that we want axes for a 3d plot
axes.plot_surface(m, c, l)                # draw a surface plot
axes.set_xlabel('m')
axes.set_ylabel('c')
plt.show()

It doesn't look like $L(m,c)$ has any nasty surprises, so let's find the minimizing $m$ and $c$.

In [ ]:
# To optimize a function of several several variables, provide them as an array
# of the appropriate length.
optpars = scipy.optimize.fmin(lambda params: loss(params[0], params[1]), x0=[0,0])
optpars

Now let's plot the data again, with the fitted straight line.

In [ ]:
# Plot the points
plt.plot(x, y, marker='o', markerfacecolor='white', markeredgecolor='black', linestyle='None')
# Plot the fitted line
def fit(x):
    return optpars[0] * x + optpars[1]
plt.plot([-3,3], [fit(-3),fit(3)], linestyle='--', color='lightblue', linewidth=2)
# Show the graphic
plt.show()

## 2.4 Simulation

TODO. _Some simple iterative simulation. Maybe ODE. Maybe an interactive graphic (taken from CUED), for the ODE for orbit around two suns (taken from NST)._

# More exercises

<div style="background-color:wheat">
These exercises in this notebook are to give you practice. They are optional, and do not contribute to your final grade. You can check your answers to labelled exercises by 
following the instructions in <a href="0.%20About%20this%20course.ipynb#0.3-Using-the-automatic-grader">&sect;0.3 Using the automatic grader</a> with 
<code style="background-color:wheat">section="notes2"</code>.
</div>

** Exercise (ex7).**
Let $x$ and $y$ be random vectors as in the [earlier exercise](#twovars). Using 
`np.atan(y/x)`,
find the angle $\theta_i$ from $(0,0)$ to each point $(x_i,y_i)$. Write your code so as to use the conventions for positive and negative angles as in the diagram below, and remember to check for $x_i=0$.

**Exercise (ex8).**
Verify that in Exercise 1 you get the same answers as `np.arctan2(y,x)`,
by computing $\text{err} = \sum (\theta_i-\phi_i)^2$ where $\phi_i$ is the answer using 
`np.arctan2`.

**Exercise (ex9).**
In [&sect;1. Programming in Python](1.%20Programming%20in%20Python.ipynb#lindley) you wrote a Pythonic simulator for a queue, based on Lindley's recursion
$$
q_{t+1} = \max(q_t+a_t-C, 0).
$$
It can be proved that this yields the same answer as
$$
q_t = q_0 + x_t - \min(0, y_t),
$$
where
$$
x_t = \sum_{u=1}^t (a_u-C)
\quad\text{and}\quad
y_t = \min_{1 \leq u \leq t} (q_0 + x_u).
$$
Compute $x=[x_1,x_2,\dots]$ using [`np.cumsum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cumsum.html).
Compute $y=[y_1,y_2,\dots]$ using 
[`np.ufunc.accumulate`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ufunc.accumulate.html).
Hence compute $q=[q_1,q_2,\dots]$.
Check your answer against your Pythonic code.